In [1]:
#!pip install -q langchain langchain-openai python-dotenv

In [2]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
print("KEY HEAD:", OPENAI_API_KEY[:7]) 

if OPENAI_API_KEY.startswith("gsk_"):
    os.environ["OPENAI_BASE_URL"] = "https://api.groq.com/openai/v1"

KEY HEAD: gsk_blU


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ✅ ChatOpenAI 그대로 사용 (Groq 키면 위에서 base URL을 Groq로 설정했으니 호환됨)
llm = ChatOpenAI(
    model="llama-3.3-70b-versatile" if OPENAI_API_KEY.startswith("gsk_") else "gpt-4o-mini",
    temperature=0.7,
    api_key=OPENAI_API_KEY,
    # base_url=os.getenv("OPENAI_BASE_URL")  # 필요 시 인자로도 가능 (환경변수 이미 설정함)
)

prompt = ChatPromptTemplate.from_template(
    """
당신은 친절한 요리사입니다.
사용자가 준 재료만 활용해서 만들 수 있는 한 가지 요리를 한국어로 추천하고,
간단한 레시피를 단계별로 써 주세요.

형식:
추천 요리: <요리명>
재료: <사용한 재료, 콤마로>
조리법:
1. ...
2. ...
3. ...

[재료] {ingredients}
"""
)

parser = StrOutputParser()

# LCEL 파이프라인
chain = prompt | llm | parser


In [6]:
user_ingredients = "계란, 밥, 김치"
result = chain.invoke({"ingredients": user_ingredients})
print(result)

추천 요리: 김치 계란밥
재료: 계란, 밥, 김치
조리법:
1. 김치를 잘게 다져서 준비합니다.
2. 계란을 풀어주세요. 소금과 후추로 간을 맞춥니다.
3. 밥에 다져놓은 김치를 넣고 계란을 넣어 잘 섞습니다.
4. 프라이팬에 기름을 두르고 열을 올려 계란이 조금 익으면 밥과 김치를 넣어 잘 볶아줍니다.
5. 완성된 김치 계란밥을 접시에 담고 시원한 국이나 다른 반찬과 함께 즐기세요!
